# Binomial Model convergence to Black-Scoles

In [1]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
#import PyPlot as plt
#using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS
using FinancialToolbox
using FinancialDerivatives
using RCall
using LsqFit

@pyinclude("Leisen_Reimer.py")

WebIO._IJuliaInit()

In [2]:
STOCK = DataFrame(CSV.File("DATA/TEVA.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2017-03-29,32.95,33.24,32.84,33.05,32.2755,4336800
2,2017-03-30,33.01,33.05,32.63,32.71,31.9435,5173500
3,2017-03-31,32.72,32.84,31.98,32.09,31.338,10839900
4,2017-04-03,32.17,32.38,31.98,32.19,31.4357,4108100
5,2017-04-04,32.77,32.8,31.95,32.0,31.2501,6617600
6,2017-04-05,32.01,32.32,31.9,31.93,31.1818,4572000
7,2017-04-06,32.0,32.39,31.91,32.35,31.5919,5701900
8,2017-04-07,32.27,32.94,31.97,32.31,31.5529,5568400
9,2017-04-10,32.51,32.79,32.36,32.51,31.7482,5158700


In [3]:
p = PlotlyJS.plot(candlestick(x    =STOCK[!, "Date" ],
                              open =STOCK[!, "Open" ],
                              high =STOCK[!, "High" ],
                              low  =STOCK[!, "Low"  ],
                              close=STOCK[!, "Close"] ),
                  Layout(title="TEVA stock Prices",
                         yaxis_title="TEVA Stock [USD]")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [4]:
CALL_1MONTH   = DataFrame(CSV.File("DATA/CALL_1MONTH.csv"  ))
CALL_3MONTHS  = DataFrame(CSV.File("DATA/CALL_3MONTHS.csv" ))
CALL_6MONTHS  = DataFrame(CSV.File("DATA/CALL_6MONTHS.csv" ))
CALL_10MONTHS = DataFrame(CSV.File("DATA/CALL_10MONTHS.csv"))

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask
,String,String,Float64,Float64,Int64,Int64
1,TEVA230120C00003000,2022-03-09 4:39PM EDT,3.0,4.98,0,0
2,TEVA230120C00004000,2022-03-09 2:47PM EDT,4.0,4.03,0,0
3,TEVA230120C00005000,2022-03-28 12:20PM EDT,5.0,3.8,0,0
4,TEVA230120C00007000,2022-03-28 3:37PM EDT,7.0,2.25,0,0
5,TEVA230120C00010000,2022-03-28 3:58PM EDT,10.0,0.85,0,0
6,TEVA230120C00012000,2022-03-28 3:42PM EDT,12.0,0.44,0,0
7,TEVA230120C00015000,2022-03-28 9:45AM EDT,15.0,0.18,0,0
8,TEVA230120C00017000,2022-03-25 11:17AM EDT,17.0,0.12,0,0
9,TEVA230120C00020000,2022-03-28 3:58PM EDT,20.0,0.08,0,0


In [33]:
#INTEREST_RATES = Dict("overnight"=>.0032871,
#                      "1 month"  =>.0044857,
#                      "3 month"  =>.0093400,
#                      "6 month"  =>.0128757,
#                      "12 month" =>.0178643)
INTEREST_RATES = [.0044514, .0098286, .0145114, .01769925]

#.0208871
S0         = STOCK.Close[length(STOCK.Close)]
Maturity1  = 1  / 12
Maturity3  = 3  / 12
Maturity7  = 6  / 12
Maturity10 = 10 / 12
Maturities = [1,3,6,10]

Strike     = Int(round(S0))
Strike10   = 10
Year_Days  = 252
#CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1]

252

In [34]:
function Return( df )
    df[!,:Return    ] =     ( df.Close - lag(df.Close, 1) ) ./ lag(df.Close, 1)
    df[!,:Return_LOG] = log.( df.Close                      ./ lag(df.Close, 1) )
end

function BINOMIAL_EU(S, K, T, r, σ, N, type_ = "call")
    Δt     = T / N
    U      = exp(σ * √Δt)
    D      = 1 / U
    #R      = ( 1 + r * Δt )
    R      = exp(r * Δt)
    q      = (R - D) / (U - D)
    
    value = 0 

    for i = 1:N+1
        node_prob = binomial(BigInt(N),BigInt(i)) * q^i * (1-q)^(N-i)
        ST        = S * U^i * D^(N-i)
        if type_ == "call"
            value += max(ST-K, 0) * node_prob
        elseif type_ == "put"
            value += max(K-ST, 0) * node_prob
        else
            throw(ValueError("type_ must be 'call' or 'put'"))
        end
    end
    return value * exp( -r * T )
end

BINOMIAL_EU (generic function with 2 methods)

In [35]:
import FinancialDerivatives.h, FinancialDerivatives.evaluate, FinancialDerivatives.Option

function h(z::T, n::Int64) where {T<:Number}
    return 0.5 + sign(z) * 0.5 * sqrt(1 - exp(-((z / (n + 1.0 / 3.0 +.1/(n+1)))^2.0) * (n + 1.0 / 6.0)))
end

function evaluate(O::Option, m::LeisenReimer, N::Int64 = 1001)
    
    if (N%2 == 1)
        N = N
    else
        N = N+1
    end
    
    Δt = O.t / N
    R  = exp(O.r * Δt)
    d1 = (log(O.s / O.k) + (O.r + O.σ * O.σ / 2) * O.t) / (O.σ * √O.t)
    d2 = d1 - O.σ * √O.t
    p  = h(d2, N)
    q  = 1 - p

    if O.call == -1
        Z = [max(0, O.k - O.s * exp((2 * i - N) * O.σ * √Δt)) for i = 0:N]
    elseif O.call == 1
        Z = [max(0, O.s * exp((2 * i - N) * O.σ * √Δt) - O.k) for i = 0:N]
    end
    
    for n = N-1:-1:0, i = 0:n
        if O.call == -1
            x = O.k - O.s * exp((2 * i - n) * O.σ * √Δt)
        elseif O.call == 1
            x = O.s * exp((2 * i - n) * O.σ * √Δt) - O.k
        end
        y = (q * Z[i+1] + p * Z[i+2]) / exp(O.r * Δt)
        Z[i+1] = max(x, y)
    end
    
    return Z[1]
end

evaluate (generic function with 28 methods)

In [36]:
Return( STOCK )

function Volatility( Returns, T, daily = false )
    VOL_DAILY  = std(skipmissing(Returns[length(Returns)-T:length(Returns)]))
    VOL_ANNUAL = VOL_DAILY * sqrt(Year_Days)
    if daily==false
        return VOL_ANNUAL
    end
    return VOL_DAILY, VOL_ANNUAL
end

VOL_ANNUAL = [Volatility( STOCK.Return, i*30 ) for i in Maturities]

REAL_CALL = [CALL_1MONTH[    CALL_1MONTH.Strike.==Strike,"Last Price"][1],
             CALL_3MONTHS[  CALL_3MONTHS.Strike.==Strike,"Last Price"][1],
             CALL_6MONTHS[  CALL_6MONTHS.Strike.==Strike,"Last Price"][1],
             CALL_10MONTHS[CALL_10MONTHS.Strike.==Strike10,"Last Price"][1],]

BINOMIAL      = Array{Float64}( undef, length(Maturities), 100 )
LEISEN_REIMER = Array{Float64}( undef, length(Maturities), 50  )
BLACK_SCHOLES = Array{Float64}( undef, length(Maturities) )

for i in 1:length(Maturities)
    #AmCall = AmericanOption(Float64(S0), Float64(Strike), Float64(INTEREST_RATES[i]), Float64(VOL_ANNUAL[i]), Float64(Maturities[i])/12., 1)
    AmCall = EuropeanOption(Float64(S0), Float64(Strike), Float64(INTEREST_RATES[i]), Float64(VOL_ANNUAL[i]), Float64(Maturities[i])/12., 1)
    
    BINOMIAL[i,:]      = [evaluate(AmCall, CoxRossRubinstein(), j) for j in 1:1:100]
    #LEISEN_REIMER[i,:] = [LeisenReimer_(S0, Strike, Maturities[i], INTEREST_RATES[i], VOL_ANNUAL[i], j)    for j in 1:2:100]
    #LEISEN_REIMER[i,:] = [evaluate(EuroCall, LeisenReimer(), j)      for j in 1:2:100]
    LEISEN_REIMER[i,:] = [py"LeisenReimerBinomial"("P", "e", "C", S0, Strike, Maturities[i]/12, INTEREST_RATES[i], INTEREST_RATES[i], VOL_ANNUAL[i], j) for j in 1:2:100]
    BLACK_SCHOLES[i]   =  evaluate(AmCall, BlackScholes())
    #BLACK_SCHOLES[i]   =  evaluate(AmCall, LongstaffSchwartz())
end

In [37]:
function Plot_BIN_BL( BINOMIAL, BL, LR, REAL, T)
    BIN_PLOT  = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=BINOMIAL,mode="markers+lines", name="Binomial Model", line=attr(color="blue", width=2, dash="dash"  ))
    BL_PLOT   = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=repeat([BL], length(BINOMIAL)), mode="lines", name="Black-Scholes prediction", line=attr(color="black", width=4, dash="dashdot"))
    LR_PLOT   = PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LR,mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" ))
    REAL_PLOT = PlotlyJS.scatter(;x=1:length(BINOMIAL), y=repeat([REAL], length(BINOMIAL)), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash"))
    layout    = Layout(title=string("TEVA Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BIN_PLOT,BL_PLOT, LR_PLOT, REAL_PLOT], layout)
    #return PlotlyJS.plot([BIN_PLOT,LR_PLOT,BL_PLOT], layout)
end

p = Plot_BIN_BL( BINOMIAL[1,:], BLACK_SCHOLES[1], LEISEN_REIMER[1,:], REAL_CALL[1], Maturities[1] )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [38]:
p = Plot_BIN_BL( BINOMIAL[2,:], BLACK_SCHOLES[2], LEISEN_REIMER[2,:], REAL_CALL[2], Maturities[2] )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [39]:
p = PlotlyJS.make_subplots(rows=2, cols=2, shared_xaxes=true, vertical_spacing=0.08, y_title="CALL price [\$]", x_title="Steps", subplot_titles=["1 Months maturity" "6 Months maturity" "15 Months maturity"; "3 Months maturity" "10 Months maturity" "22 Months maturity"])

r = c = 1
for i in 1:length(Maturities)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=BINOMIAL[i,:],mode="markers+lines", line=attr(color="blue", width=2)), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([BLACK_SCHOLES[i]], length(BINOMIAL[i,:])), mode="lines", line=attr(color="black", width=3, dash="dashdot")), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LEISEN_REIMER[i,:],mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" )), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([REAL_CALL[i]], length(BINOMIAL[i,:])), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash")), row=r, col=c)
    c+=1
    if c == 3
        c  = 1
        r += 1
    end
end
    
p.plot.layout.annotations[5].yshift=-15
p.plot.layout.annotations[6].xshift=-22
PlotlyJS.relayout!(p, title_text="TEVA Call Price, Strike ~ S0, as function of number of steps for several maturities", showlegend=false, width=1000, height=1000)
#print(json(p.plot.layout.annotations, 2))
p

data: [
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, showlegend, template, title, width, xaxis, xaxis2, xaxis3, xaxis4, yaxis, yaxis2, yaxis3, and yaxis4"

In [46]:
p = PlotlyJS.make_subplots(rows=2, cols=2, shared_xaxes=true, vertical_spacing=0.08, y_title="CALL price [\$]", x_title="Steps", subplot_titles=["1 Months maturity" "6 Months maturity" "15 Months maturity"; "3 Months maturity" "10 Months maturity" "22 Months maturity"])

r = c = 1
for i in 1:length(Maturities)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=BINOMIAL[i,:],mode="markers+lines", line=attr(color="blue", width=2)), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([BLACK_SCHOLES[i]], length(BINOMIAL[i,:])), mode="lines", line=attr(color="black", width=3, dash="dashdot")), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:2:length(BINOMIAL),    y=LEISEN_REIMER[i,:],mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" )), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([REAL_CALL[i]], length(BINOMIAL[i,:])), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash")), row=r, col=c)
    r+=1
    if r == 3
        r  = 1
        c += 1
    end
end
    
p.plot.layout.annotations[5].yshift=-13
p.plot.layout.annotations[6].xshift=-22
PlotlyJS.relayout!(p, title_text="TEVA Call Price, Strike ~ S0, as function of number of steps for several maturities", showlegend=false, width=1000, height=800)
#print(json(p.plot.layout.annotations, 2))
p

data: [
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, showlegend, template, title, width, xaxis, xaxis2, xaxis3, xaxis4, yaxis, yaxis2, yaxis3, and yaxis4"

In [13]:
S = K = 100.
r = .01
σ = 0.2
T = 1.
euro_call = EuropeanOption(S, K, r, σ, T, 1)

#bs = blsprice(S, K, r, T, σ)
#bm = [BINOMIAL_EU(S, K, T, r, σ, N) for N in 1:1000]
bs =  evaluate(euro_call, BlackScholes())
bm = [evaluate(euro_call, CoxRossRubinstein(), N) for N in 1:1:500]
#lr = [evaluate(euro_call, LeisenReimer(),      N) for N in 1:2:1000]
lr = [py"LeisenReimerBinomial"("P", "e", "C", S, K, T, r, r, σ, N) for N in 1:2:500]

250-element Vector{Float64}:
 8.265444950977374
 8.403232838109828
 8.42093430274055
 8.426569223379763
 8.429071383442764
 8.430399782905585
 8.431189275248071
 8.431696630815647
 8.432041971526694
 8.43228764143732
 8.432468628853853
 8.432605808162036
 8.432712261804822
 ⋮
 8.433316926612692
 8.433316941293437
 8.433316955778158
 8.433316970089304
 8.433316984218019
 8.433316998181288
 8.433317011962012
 8.43331702560406
 8.43331703905138
 8.43331705233945
 8.433317065463873
 8.433317078461519

In [14]:
function Plot_BIN_BL( BINOMIAL, BL, LR, T, xrange=-1, yrange=-1, bin = true)
    if bin == true
        BIN_PLOT  = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=BINOMIAL,mode="markers+lines", name="Binomial Model", line=attr(color="blue", width=2, dash="dash"  ))
    end
    BL_PLOT   = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=repeat([BL], length(BINOMIAL)), mode="lines", name="Black-Scholes prediction", line=attr(color="black", width=4, dash="dashdot"))
    LR_PLOT   = PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LR,mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" ))
    
    if yrange!=-1 && xrange!=-1
        layout    = Layout(title=string("Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                           yaxis_title="CALL price [\$]", xaxis_title="Steps", xaxis_range=xrange, yaxis_range=yrange )
    elseif xrange!=-1
        layout    = Layout(title=string("Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                           yaxis_title="CALL price [\$]", xaxis_title="Steps",xaxis_range=xrange)
    elseif yrange!=-1
        layout    = Layout(title=string("Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                           yaxis_title="CALL price [\$]", xaxis_title="Steps",yaxis_range=yrange)
    else
        layout    = Layout(title=string("Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                           yaxis_title="CALL price [\$]", xaxis_title="Steps")
    end
    if bin == true
        return PlotlyJS.plot([BIN_PLOT,BL_PLOT, LR_PLOT], layout)
    end
    return PlotlyJS.plot([BL_PLOT, LR_PLOT], layout)
end

Plot_BIN_BL( bm, bs, lr, 12)

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [15]:
Plot_BIN_BL( bm, bs, lr, 12, [0,30],-1, false )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [16]:
function chi(Exp, Obs)
    return sum((Exp.-Obs).^2 ./ Exp)
end

chi (generic function with 1 method)

In [17]:
model(t, p) = bs .+ ( p ./ t )
tdata = collect(1:2:500)
ydata = bm[tdata]
p0 = [0.5]

fit = curve_fit(model, tdata, ydata, p0)
param = fit.param
print(param)


BIN_PLOT  = PlotlyJS.scatter(;x=1:length(bm),   y=bm,mode="markers+lines", name="CRR", line=attr(color="blue", width=2, dash="dash"  ))
converg   = PlotlyJS.scatter(;x=tdata, y= model(tdata, param[1]), mode="markers+lines", name="BS + const / n", line=attr(color="black", width=2, dash="solid"  ))

layout    = Layout(title="Convergence of CCR",
                         yaxis_title="CALL price [\$]", xaxis_title="Steps",xaxis_range=[-2,100])

PlotlyJS.plot([BIN_PLOT, converg], layout)

[1.9841032478365126]

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [18]:
model(t, p) = bs .+ ( p ./ t.^2 )
tdata = collect(1:250)
ydata = lr
p0 = [0.5]

fit = curve_fit(model, tdata, ydata, p0)
param = fit.param
print(param)

LR_PLOT  = PlotlyJS.scatter(;x=1:2:length(bm),   y=lr,mode="markers+lines", name="LR", line=attr(color="red", width=2, dash="dash"  ))
converg   = PlotlyJS.scatter(;x=1:2:length(bm), y= model(tdata, param[1]), mode="markers+lines", name="BS + const / n^2", line=attr(color="black", width=2, dash="solid"  ))

layout    = Layout(title="Convergence of LR",
                         yaxis_title="CALL price [\$]", xaxis_title="Steps",xaxis_range=[-2,100])

PlotlyJS.plot([LR_PLOT, converg], layout)

[-0.16406214726245952]

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [19]:
chi(lr, model(tdata, param[1]))

2.296312473313388e-5

In [44]:
PlotlyJS.plot(CALL_10MONTHS.Strike, CALL_10MONTHS."Last Price")

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"